In [1]:
import time
import torch



from modules.Dcls import Dcls1d


import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

assert torch.cuda.is_available()
cuda_device = torch.device("cuda")  # device object representing GPU

batch = 1
size_sig = 16
in_channels = 1
out_channels = 1
kernel_size = 4
dilation = 2
stride = 1
padding = 0
groups = 1
bias = False

m = torch.nn.Conv1d(in_channels=in_channels,
              out_channels=out_channels,
              kernel_size=kernel_size,
              dilation=dilation,
              stride=stride,
              padding=padding,
              groups=groups,
              bias=bias).to(cuda_device)
n = Dcls1d(in_channels=in_channels,
              out_channels=out_channels,
              kernel_size=kernel_size,
              dilation=dilation,
              stride=stride,
              padding=padding,
              groups=groups,
              bias=bias).to(cuda_device)

m.weight = torch.nn.Parameter(
                      torch.tensor([[[20., 40.,60., 80.]]],device=cuda_device),
                     requires_grad = True)
n.weight = torch.nn.Parameter(
                      torch.tensor([[[20., 40.,60., 80.]]],device=cuda_device),
                     requires_grad = True)
X1 = torch.nn.Parameter(
                      torch.tensor([[[1., 2., 3., 4., 5., 6., 7., 8., 9., 10., 11., 12.,13., 14., 15., 16.]]],device=cuda_device),
                      requires_grad = True) 
X2 = torch.nn.Parameter(
                      torch.tensor([[[1., 2., 3., 4., 5., 6., 7., 8., 9., 10., 11., 12.,13., 14., 15., 16.]]],device=cuda_device),
                      requires_grad = True) 
print(m(X1))
print(n(X2))

back_truth = torch.nn.Parameter(
                      torch.tensor([[[1., 2., 3., 4., 5., 6., 7., 8., 9., 10.]]],device=cuda_device),
                     requires_grad = True)

var2 = (n(X2) - back_truth).norm()
var1 = (m(X1) - back_truth).norm()

var1.backward();
var2.backward();

print(m.weight.grad) 
print(n.weight.grad)

print(X1.grad) 
print(X2.grad)

ModuleNotFoundError: No module named 'modules'

In [ ]:
batch = 256
in_channels = 512
out_channels = 1024
stride = 1
padding = 0
dilation = 4
kernel_size = 3
img_size = 2240
bias = False
groups = 1
    

m = torch.nn.Conv1d(in_channels=in_channels,
              out_channels=out_channels,
              kernel_size=kernel_size,
              dilation=dilation,
              stride=stride,
              padding=padding,
              groups=groups,
              bias=bias).to(cuda_device)



n = Dcls1d(in_channels=in_channels,
              out_channels=out_channels,
              kernel_size=kernel_size,
              dilation=dilation,
              stride=stride,
              padding=padding,
              groups=groups,
              bias=bias).to(cuda_device)
n.weight = m.weight



Y = torch.nn.Parameter(torch.randn((batch,in_channels,img_size),device=cuda_device))

In [ ]:
forward = 0
backward = 0
for _ in range(10):
    start = time.time()
    c = m(Y)
    torch.cuda.synchronize()
    forward += time.time() - start

    start = time.time()
    (c.sum() ).backward()
    torch.cuda.synchronize()
    backward += time.time() - start

print('Forward: {:.3f} us | Backward {:.3f} us'.format(forward * 1e6/1e1, backward * 1e6/1e1))

In [ ]:
forward = 0
backward = 0
for _ in range(10):
    start = time.time()
    d = n(Y)
    torch.cuda.synchronize()
    forward += time.time() - start

    start = time.time()
    (d.sum() ).backward()
    torch.cuda.synchronize()
    backward += time.time() - start

print('Forward: {:.3f} us | Backward {:.3f} us'.format(forward * 1e6/1e1, backward * 1e6/1e1))